In [ ]:
import pandas as pd
import numpy as np
import scipy
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances

import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import random
from scipy import sparse
from scipy.stats import pearsonr
import re

In [ ]:
from  google.colab  import  drive 
drive.mount ( '/content/drive' )

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/reviews_with_emotions_full.csv')
df.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [ ]:
df

,RestaurantId,Name,UserId,Review,Rating,Emotion
0,1,aleria,1,A full 5 stars! Phenomenal Michelin ranked res...,5,admiration
1,1,aleria,2,We spent great evening here. Cozy restaurant w...,5,admiration
2,1,aleria,3,10/10. Aleria was our highlight during our tri...,5,admiration
3,1,aleria,4,Great service and fantastic food. We enjoyed e...,5,admiration
4,1,aleria,5,Let me put it his way: simply perfect. We hate...,5,admiration
...,...,...,...,...,...,...
26203,30,ydria,1000,"Very good grilled octopus, spinach pie and Sau...",5,admiration
26204,30,ydria,1001,"The food is sensarional, the service is great....",5,admiration
26205,30,ydria,1002,Risotto was excellent. Wine was an extremely s...,4,admiration
26206,30,ydria,1003,We decided to eat here for an afternoon snack ...,5,joy


In [ ]:
username = input ("Username: ")
password = input ("Password: ")

Username: 1000
Password: 5


# STEP 1 - Extract User's Past Comments


In [ ]:
users_comments = df[df['UserId'] == int(username)]

In [ ]:
users_comments

,RestaurantId,Name,UserId,Review,Rating,Emotion
2553,3,arcadia,1000,This is one more restaurant on this touristic ...,2,neutral
6827,8,efcharis,1000,"Excellent food here, we saw this restaurant wh...",5,admiration
8827,10,gods_restaurant,1000,We read some great reviews before visiting God...,5,approval
9832,11,karamanlidika,1000,This place feels like a unique restaurant expe...,5,admiration
11390,13,liondi,1000,We had a reservation and when we arrived just ...,1,realization
12395,14,lithos,1000,We went there because of the many positive rev...,5,admiration
13400,15,maiandros,1000,I just can't say enough of how amazing the foo...,5,admiration
16597,19,oroscopo,1000,We visited there today and the service is grea...,5,admiration
17602,20,o_thanasis,1000,"After seeing the archaeological sites nearby, ...",5,admiration
20731,24,smile,1000,If you are looking for a family friendly place...,5,admiration


# STEP 2 - Extract the Emotions from the comments

In [ ]:
users_comments

,RestaurantId,Name,UserId,Review,Rating,Emotion
2553,3,arcadia,1000,This is one more restaurant on this touristic ...,2,neutral
6827,8,efcharis,1000,"Excellent food here, we saw this restaurant wh...",5,admiration
8827,10,gods_restaurant,1000,We read some great reviews before visiting God...,5,approval
9832,11,karamanlidika,1000,This place feels like a unique restaurant expe...,5,admiration
11390,13,liondi,1000,We had a reservation and when we arrived just ...,1,realization
12395,14,lithos,1000,We went there because of the many positive rev...,5,admiration
13400,15,maiandros,1000,I just can't say enough of how amazing the foo...,5,admiration
16597,19,oroscopo,1000,We visited there today and the service is grea...,5,admiration
17602,20,o_thanasis,1000,"After seeing the archaeological sites nearby, ...",5,admiration
20731,24,smile,1000,If you are looking for a family friendly place...,5,admiration


# STEP 3 - Find the most common positive one

In [ ]:
user_positive_comments = users_comments[users_comments['Rating'] > 3]

In [ ]:
user_positive_comments

,RestaurantId,Name,UserId,Review,Rating,Emotion
6827,8,efcharis,1000,"Excellent food here, we saw this restaurant wh...",5,admiration
8827,10,gods_restaurant,1000,We read some great reviews before visiting God...,5,approval
9832,11,karamanlidika,1000,This place feels like a unique restaurant expe...,5,admiration
12395,14,lithos,1000,We went there because of the many positive rev...,5,admiration
13400,15,maiandros,1000,I just can't say enough of how amazing the foo...,5,admiration
16597,19,oroscopo,1000,We visited there today and the service is grea...,5,admiration
17602,20,o_thanasis,1000,"After seeing the archaeological sites nearby, ...",5,admiration
20731,24,smile,1000,If you are looking for a family friendly place...,5,admiration
21736,25,strofi,1000,"Reservations strongly recommended, especially ...",5,admiration
26203,30,ydria,1000,"Very good grilled octopus, spinach pie and Sau...",5,admiration


In [ ]:
already_visited = user_positive_comments["Name"].values

In [ ]:
already_visited

array(['efcharis', 'gods_restaurant', 'karamanlidika', 'lithos',
       'maiandros', 'oroscopo', 'o_thanasis', 'smile', 'strofi', 'ydria'],
      dtype=object)

In [ ]:
user_positive_comments = user_positive_comments.groupby("Emotion").count()
user_positive_comments = user_positive_comments.sort_values(by=['Emotion'], ascending = True)

In [ ]:
user_positive_comments

,RestaurantId,Name,UserId,Review,Rating
Emotion,,,,,
admiration,9,9,9,9,9
approval,1,1,1,1,1


In [ ]:
most_felt_emotion = user_positive_comments.index.values[0]

In [ ]:
most_felt_emotion

'admiration'

# STEP 4 - Find rows of the dataset with the same emotion

In [ ]:
possible_options = df[df['Emotion'] == most_felt_emotion]

In [ ]:
possible_options

,RestaurantId,Name,UserId,Review,Rating,Emotion
0,1,aleria,1,A full 5 stars! Phenomenal Michelin ranked res...,5,admiration
1,1,aleria,2,We spent great evening here. Cozy restaurant w...,5,admiration
2,1,aleria,3,10/10. Aleria was our highlight during our tri...,5,admiration
3,1,aleria,4,Great service and fantastic food. We enjoyed e...,5,admiration
4,1,aleria,5,Let me put it his way: simply perfect. We hate...,5,admiration
...,...,...,...,...,...,...
26202,30,ydria,999,Fine dining outdoors with great service. It w...,5,admiration
26203,30,ydria,1000,"Very good grilled octopus, spinach pie and Sau...",5,admiration
26204,30,ydria,1001,"The food is sensarional, the service is great....",5,admiration
26205,30,ydria,1002,Risotto was excellent. Wine was an extremely s...,4,admiration


# STEP 5 - Find restaurants that provides the feeling the most and the user has not visited yet.

In [ ]:
recommends_df = possible_options.groupby("Name")['Emotion'].count().to_frame()
recommends_df = recommends_df.sort_values(by=['Emotion'], ascending = False)

In [ ]:
recommends_df

,Emotion
Name,
balcony,835
lithos,833
efcharis,818
liondi,818
maiandros,809
oroscopo,793
arcadia,777
elaea,767
karamanlidika,756


In [ ]:
possible_recommendations = recommends_df.index.values

In [ ]:
possible_recommendations

array(['balcony', 'lithos', 'efcharis', 'liondi', 'maiandros', 'oroscopo',
       'arcadia', 'elaea', 'karamanlidika', 'strofi', 'sin_athina',
       'aleria', 'to_paradosiako', 'smile', 'gods_restaurant', 'nolan',
       'ydria', 'to_kati_allo', 'tzitzikas_kai_mermigas', 'o_thanasis',
       'mironi', 'alexander_the_great', 'scholarchio', 'klimataria',
       'rozalia', 'atitamos', 'mani_mani', 'athenascook'], dtype=object)

# STEP 6 - Recommend the restaurants

In [ ]:
possible_recommendations

array(['balcony', 'lithos', 'efcharis', 'liondi', 'maiandros', 'oroscopo',
       'arcadia', 'elaea', 'karamanlidika', 'strofi', 'sin_athina',
       'aleria', 'to_paradosiako', 'smile', 'gods_restaurant', 'nolan',
       'ydria', 'to_kati_allo', 'tzitzikas_kai_mermigas', 'o_thanasis',
       'mironi', 'alexander_the_great', 'scholarchio', 'klimataria',
       'rozalia', 'atitamos', 'mani_mani', 'athenascook'], dtype=object)

In [ ]:
already_visited

array(['efcharis', 'gods_restaurant', 'karamanlidika', 'lithos',
       'maiandros', 'oroscopo', 'o_thanasis', 'smile', 'strofi', 'ydria'],
      dtype=object)

In [ ]:
recommendations = []

In [ ]:
for i in possible_recommendations:
  if i not in already_visited:
    recommendations.append(i)

In [ ]:
recommendations

['balcony',
 'liondi',
 'arcadia',
 'elaea',
 'sin_athina',
 'aleria',
 'to_paradosiako',
 'nolan',
 'to_kati_allo',
 'tzitzikas_kai_mermigas',
 'mironi',
 'alexander_the_great',
 'scholarchio',
 'klimataria',
 'rozalia',
 'atitamos',
 'mani_mani',
 'athenascook']